# Competition Description
Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

# DecisionTree Regressor of Housing Price Competition
Link to competition: \ submitted by: Juan Fakri

This code example uses sklearn Linear_model to predict the price of a house based on some characteristics of the house.

Here are some strategies to solve the problem:

1. Import library and get data
2. clean up data
3. Build, fit, and evaluate Sklearn linear regression models that perform predictions.
4. Run hyperparameter tuning
5. submit your work 



### 1.   Import the Libraries and get the Data

In [58]:
# Author: Juan Pablo Contreras
# This is the notebook used for my submission for the House Prices - Advanced Regression Techniques competition

import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

In [59]:
# read the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [60]:
train_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


## 2. Data Cleaning

NaN values ​​in the dataset:
Yes| Categorical values ​​in the dataset:
Yes| Numbers in dataset:
Yes The only difference between the training dataset and the test dataset is that the test dataset does not contain the SalePrice column. To clear the data, do the following: 

- 2.1. The SalePrice column is removed from the training dataset to match the columns of the training and test DFs. 
- 2.2. The test DF is added to the training DF, so we can perform cleanup on both the training DF and the test DF.
- 2.3. Categorical values ​​are hot encoded using pands get_dummies() function 
- 2.4. NaN values ​​are converted to -1.
- 2.5. The dataset is again split into training and testing.
- 2.6. The training dataset is split into two parts (training and validation) to see how well the trained model performs. 
- 2.7. Data are normalized using the training data. Validation and test data are transformed using fitting parameters for normalization of training data. 




### 2.1 Delete SalePrice from Training Dataset



In [61]:
# find number of rows
num_rows_train = len(train_data.index)
print("num rows in training data: " + str(num_rows_train))
print("num rows in test data: " + str(len(test_data.index)))

# store and drop the training predictions
y_train = train_data.iloc[:,-1]
train_data.drop(columns=train_data.columns[-1], 
        axis=1, 
        inplace=True)

num rows in training data: 1460
num rows in test data: 1459


### 2.2 Merge Train and Test Data

In [62]:
# merge all data for cleaning
all_data = train_data.append(test_data, ignore_index=True)
print("num rows in all data: " + str(len(all_data.index)))

# delete id column because ID is not a predictor of house price
ids = all_data.drop('Id', axis=1)

num rows in all data: 2919


C:\Users\ASUS\AppData\Local\Temp\ipykernel_17624\479556246.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = train_data.append(test_data, ignore_index=True)


### 2.3 & 2.4 Handle Categorical and NaN values

In [63]:
# convert categorical data to numerical (0 or 1)
all_data = pd.get_dummies(data=all_data)

# replace NaN values
all_data = all_data.fillna(-1)

all_data.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,...,0,0,0,1,0,0,0,0,1,0


In [64]:
all_data.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,...,0,0,0,1,0,0,0,0,1,0


In [65]:
all_data.columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=289)

In [66]:
def missing_value_describe(data):
# Check for missing values in data
    missing_value_stats = (data.isnull().sum() / len(data)*100)
    missing_value_col_count = sum(missing_value_stats > 0)
    missing_value_stats = missing_value_stats.sort_values(ascending=False)[:missing_value_col_count]
    print("Number of columns with missing values:", missing_value_col_count)
    if missing_value_col_count != 0:
# Print column names with percentage of missing values
        print("\nPercentage lost (decreasing):")
        print(missing_value_stats)
    else:
        print("No data is lost!!!")
missing_value_describe(all_data)

Number of columns with missing values: 0
No data is lost!!!


### 2.5 Split data back into Train, and Test

In [67]:
# split data back into train and test data
X_train = all_data.iloc[0:num_rows_train,:]
print("num rows in X train: " + str(len(X_train.index)))
print("num rows in y train: " + str(len(y_train.index)))
X_test = all_data.iloc[num_rows_train:,:]
print("num rows in X test: " + str(len(X_test.index)))

print("Training data features head: ")
print(X_train.head())
print()
print("Training data predictions head: ")
print(y_train.head())
print()
print("Test data features head:")
print(X_test.head())

num rows in X train: 1460
num rows in y train: 1460
num rows in X test: 1459
Training data features head: 
   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  SaleType_ConLw  SaleType_New  \
0          2003       196.0       706.0  ...               0             0   
1          1976         0.0       978.0  ...               0             0   
2          2002       162.0       486.0  ...               0             0   
3          1970         0.0       216.0  ...               0             0   
4          2000       

### 2.6 Split train data into train and validation
The new training data will be 80% of the train dataset, and the validation data 20% of the train data set

In [68]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [69]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_val_transf = scaler.transform(X_val)
X_test_transf = scaler.transform(X_test)

### 3. Build, fit, and evaluate linear regression models
A sklearn linear regression model is used.

1. The model is built using the sklearn library
2. The model is fitted using the training data
3. The model is validated against validation data
4. Predictions are made based on test data 

In [70]:
# linear model
regr = DecisionTreeRegressor()
# fit model using training data
regr.fit(X_train, y_train)

DecisionTreeRegressor()

### 3.3 Evaluating the Model

<b>How good did the model fit the training data?</b>

First, find the R^2 value of the training data predictions to see how well the linear model fits the trained data. 

In [75]:
r2_train = regr.score(X_train,y_train)
print("R^2 on training data: " + str(r2_train))

R^2 on training data: 1.0


The model fit is very good, reaching a maximum value of R^2 which is 1.  Saying that the linear model fits the training data perfectly. Let's hope this model works well on untrained data!

<b>How good is the model on new data?</b>

Now that you know how your model behaves on training data, you want to know how it behaves on data you've never seen before. Use the score() function to make predictions on validation data and evaluate how good those predictions are. 

In [76]:
r2_validation = regr.score(X_val,y_val)
print("R^2 on validation data: " + str(r2_validation))

R^2 on validation data: 0.7731669023661499


The R^2 value in the validation dataset is about 0.773, good enough for my heart's feelings... Let's see we'll predict the sale price of a home in a validation data set and then see how far this prediction is from the actual sale price

In [81]:
# predict prices on the validation data
pred_val = regr.predict(X_val)
print("Five first predictions: ")
print(pred_val[:5])
print()

Five first predictions: 
[155000. 430000. 108000. 159000. 285000.]



In [82]:
# Compare the Predicted values on the validation data set to the actual SalePrices of the validation data set using the root mean squared error
MSE = np.square(np.subtract(y_val,pred_val)).mean()   
   
rsme = math.sqrt(MSE)  
print("Root Mean Square Error:\n")  
print(rsme)

Root Mean Square Error:

41711.91500564744


This roughly means that on average, the predicted sale price of a house drops by $41,711 This error is quite big! This prediction will probably never be used in the real world. Therefore, we need to do some hyper parameter tuning to make the model better at predicting house prices